In [1]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [2]:
df = pd.read_csv('AQI_clean.csv')
df.head()

,country,city,aqi_value,aqi_category,co_aqi_value,co_aqi_category,ozone_aqi_value,ozone_aqi_category,no2_aqi_value,no2_aqi_category,pm2_5_aqi_value,pm2_5_aqi_category,latitude,longitude
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate,44.7444,44.2031
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
2,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869
3,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate,37.1667,15.1833
4,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good,53.0167,20.8833


In [3]:
engine = create_engine("sqlite:///aqi.sqlite")

In [4]:
query = """CREATE TABLE "aqi" (
	"id" integer Primary key,
	"country" varchar(100),
	"city" varchar(100),
	"aqi_value" integer,
	"aqi_category" varchar(30),
	"co_aqi_value" integer,
	"co_aqi_category" varchar(30),
	"ozone_aqi_value" integer,
	"ozone_aqi_category" varchar(30),
	"no2_aqi_value" integer,
	"no2_aqi_category" varchar(30),
	"pm2_5_aqi_value" integer,
	"pm2_5_aqi_category" varchar(30),
	"latitude" float,
	"longitude" float
);"""

with engine.connect() as conn:
    conn.execute(text(query))

In [5]:
df.to_sql("aqi", con=engine, index=False, method="multi", if_exists="append")

16393

In [6]:
 # INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print() 

aqi
-----------
id INTEGER
country VARCHAR(100)
city VARCHAR(100)
aqi_value INTEGER
aqi_category VARCHAR(30)
co_aqi_value INTEGER
co_aqi_category VARCHAR(30)
ozone_aqi_value INTEGER
ozone_aqi_category VARCHAR(30)
no2_aqi_value INTEGER
no2_aqi_category VARCHAR(30)
pm2_5_aqi_value INTEGER
pm2_5_aqi_category VARCHAR(30)
latitude FLOAT
longitude FLOAT



In [7]:
query = """
    SELECT *
    FROM aqi
    WHERE country = "Brazil"
"""

df2 = pd.read_sql(text(query), con=engine)
df2.head()

,id,country,city,aqi_value,aqi_category,co_aqi_value,co_aqi_category,ozone_aqi_value,ozone_aqi_category,no2_aqi_value,no2_aqi_category,pm2_5_aqi_value,pm2_5_aqi_category,latitude,longitude
0,2,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-5.2900,-44.4900
1,3,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good,-11.2958,-41.9869
2,16,Brazil,Pinheiral,154,Unhealthy,5,Good,0,Good,13,Good,154,Unhealthy,-22.5128,-44.0008
3,24,Brazil,Porecatu,30,Good,1,Good,9,Good,2,Good,30,Good,-22.7558,-51.3789
4,31,Brazil,Pouso Alegre,32,Good,1,Good,7,Good,2,Good,32,Good,-22.2281,-45.9336


In [8]:
 # Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(autoload_with=engine)

# Print all of the classes mapped to the Base

print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.aqi'>]
